In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### Segédfüggvények

In [47]:
def limitalt_dummy_valtozo(indf, oszlop, limit, drop_last=True):
    uj_valtozok = []
    df = indf.copy()
    egyedi_elemszam=df[oszlop].nunique()
    if egyedi_elemszam <= limit and drop_last is True:
        egyedi_elemszam = egyedi_elemszam - 1
    tmp = df[oszlop].value_counts()[:limit]
    for value in list(tmp.index):
        def f(x):
            if x == value:
                return 1
            else:
                return 0
        df[oszlop+'='+str(value)] = df[oszlop].apply(f)
        uj_valtozok.append(oszlop + '=' + str(value))
    return df, uj_valtozok

In [48]:
def myxval(model,train_df,bemeno_valtozok,fold_num=3):
    train_df['xval']=train_df['ID']%fold_num
    auc_list=[]
    for i in range(fold_num):
        mini_train=train_df[train_df['xval']!=i].copy()
        mini_test=train_df[train_df['xval']==i].copy()
        model.fit(mini_train[bemeno_valtozok],mini_train['TARGET'])
        mini_test['p1']=model.predict_proba(mini_test[bemeno_valtozok])[:,1]
        auc=roc_auc_score(mini_test['TARGET'],mini_test['p1'])
        auc_list.append(auc)
    return np.mean(auc_list)

In [49]:
def myxval_predict(model,train_df,bemeno_valtozok,fold_num=3):
    train_df['xval']=train_df['ID']%fold_num
    auc_list=[]
    for i in range(fold_num):
        mini_train=train_df[train_df['xval']!=i].copy()
        mini_test=train_df[train_df['xval']==i].copy()
        model.fit(mini_train[bemeno_valtozok],mini_train['TARGET'])
        mini_test['p1']=model.predict(mini_test[bemeno_valtozok])
        auc=roc_auc_score(mini_test['TARGET'],mini_test['p1'])
        auc_list.append(auc)
    return np.mean(auc_list)

### --------------------------------------------------------------------- ###

In [50]:
train_data = pd.read_csv('./data/public_train.csv')
test_data = pd.read_csv('./data/public_test.csv')

C:\Programs\miniconda\envs\dap2020_jupyter\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [51]:
# Train or test?
train_data['train_or_test'] = 'train'
test_data['train_or_test'] = 'test'

In [52]:
train_data['TARGET'].dtype

dtype('int64')

In [53]:
df = pd.concat([train_data, test_data], sort=True)

In [54]:
df['TARGET'].dtypes

dtype('float64')

In [55]:
# Hianyzo adatok kitoltese
for att in df.columns:
    if att=="TARGET":
        pass
    #amik nem számok (string értékek stb...)
    elif df[att].dtype=="object": 
        #print(att)
        df[att]=df[att].fillna("nincs_adat")
    else:
        #test és train közös mediánja
        #ha van benne NaN, akkor azzal töltötte volna fel, de ez nem jó, ezért kell a nanmedian függvény
        #de nem a mediánra kéne itt kitölteni az fix!!!
        #df[att]=df[att].fillna(0) ez is lehetne
        df[att]=df[att].fillna(0)

In [56]:
# float64 to int64
for att in df.columns:
    if att =="TARGET":
        pass
    elif df[att].dtype == "float64":
        df[att] = df[att].astype('int64')

In [57]:
df.columns

Index(['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'BIRTH_CITY',
       'BIRTH_STATE', 'CARS_FLAG', 'CONF_CITIZEN_CARD_NUM', 'CONF_TAX_STATUS',
       'C_NAME_GIVEN', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'ID', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH',
       'PROF_CITY', 'PROF_CODE', 'PROF_PHONE_AREA_CODE', 'PROF_PHONE_GIVEN',
       'PROF

In [58]:
bemeno_valtozok = ['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'CARS_FLAG', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH', 'P_ADDRESS_TYPE', 'P_ASSETS_VALUE',
       'P_MONTHLY_INCOME', 'RES_BOROUGH', 'RES_TYPE', 'SEX', 'S_FLAG',
       'VISA_FLAG']

In [59]:
object_valtozok = []
uj_valtozok = []
for att in bemeno_valtozok:
    if df[att].dtype == 'object':
        df, val = limitalt_dummy_valtozo(df, att, 6)
        object_valtozok.append(att)
        uj_valtozok.extend(val)

In [60]:
bemeno_valtozok = bemeno_valtozok + uj_valtozok
bemeno_valtozok = [val for val in bemeno_valtozok if val not in object_valtozok]

In [61]:
d = []
for att in bemeno_valtozok:
    d.append(df[att].dtype)
d

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'

## Modellezés

In [62]:
train_df = df[df['train_or_test'] == 'train'].copy()
test_df = df[df['train_or_test'] == 'test'].copy()

In [63]:
train_df.columns

Index(['AGE', 'AME_FLAG', 'APP_SUB_TYPE', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MONTH_4',
       'BILL_AMOUNT_MONTH_5', 'BILL_AMOUNT_MONTH_6', 'BIRTH_CITY',
       'BIRTH_STATE', 'CARS_FLAG', 'CONF_CITIZEN_CARD_NUM', 'CONF_TAX_STATUS',
       'C_NAME_GIVEN', 'C_PRODUCT', 'DAY_OF_PAYMENT', 'EDU_LEVEL',
       'EMAIL_FLAG', 'HOME_PHONE_FLAG', 'ID', 'L_BALANCE', 'MATE_PROF_CODE',
       'MCARD_FLAG', 'MONTHS_IN_RES', 'M_IN_THE_JOB', 'M_STATUS', 'NAT',
       'NUMBERO_OF_BANK_ACCOUNTS', 'NUMBER_OF_DEPENDANTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'OCARDS_FLAG', 'OCC_TYPE',
       'O_INCOMES', 'PAY_AMOUNT_MONTH_1', 'PAY_AMOUNT_MONTH_2',
       'PAY_AMOUNT_MONTH_3', 'PAY_AMOUNT_MONTH_4', 'PAY_AMOUNT_MONTH_5',
       'PAY_AMOUNT_MONTH_6', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3',
       'PAY_MONTH_4', 'PAY_MONTH_5', 'PAY_MONTH_6', 'PROF_BOROUGH',
       'PROF_CITY', 'PROF_CODE', 'PROF_PHONE_AREA_CODE', 'PROF_PHONE_GIVEN',
       'PROF

In [64]:
d = []
for att in bemeno_valtozok:
    d.append(train_df[att].dtype)
d

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'

In [65]:
estimators = list(range(150, 301, 25))

In [66]:
depth = list(range(5, 16, 5))

In [67]:
estimators

[150, 175, 200, 225, 250, 275, 300]

In [69]:
from sklearn.ensemble import RandomForestClassifier
opt_rfc = dict()
for d in depth:
    for e in estimators:
        model = RandomForestClassifier(random_state=42, n_estimators=e, max_depth=d)
        auc = myxval(model, train_df, bemeno_valtozok)
        opt_rfc[(d, e)] = auc

In [70]:
opt_rfc

{(5, 150): 0.6827286062409773,
 (5, 175): 0.6831725543853353,
 (5, 200): 0.6832803759203486,
 (5, 225): 0.6832729107808468,
 (5, 250): 0.6833498716412952,
 (5, 275): 0.6833830958449284,
 (5, 300): 0.6833114612370377,
 (10, 150): 0.724571678031045,
 (10, 175): 0.7242235250871558,
 (10, 200): 0.7241145697102674,
 (10, 225): 0.7243247982920159,
 (10, 250): 0.7246694694470724,
 (10, 275): 0.7248942000560218,
 (10, 300): 0.7248263033160015,
 (15, 150): 0.7415006473169733,
 (15, 175): 0.7419665397328307,
 (15, 200): 0.7421035895681879,
 (15, 225): 0.7424823253984815,
 (15, 250): 0.7429615543712916,
 (15, 275): 0.7430576293437952,
 (15, 300): 0.7431940836655578}

In [71]:
from sklearn.ensemble import GradientBoostingClassifier
opt_gbc = dict()
for d in depth:
    for e in estimators:
        model = GradientBoostingClassifier(random_state=42, n_estimators=e, max_depth=d)
        auc = myxval(model, train_df, bemeno_valtozok)
        opt_gbc[(d, e)] = auc

In [72]:
opt_gbc

{(5, 150): 0.7342727429671605,
 (5, 175): 0.7361634051967485,
 (5, 200): 0.7374226991794907,
 (5, 225): 0.7393877539048163,
 (5, 250): 0.7404239331288719,
 (5, 275): 0.7411828367634365,
 (5, 300): 0.7420622826647314,
 (10, 150): 0.7438327695592072,
 (10, 175): 0.7440240936484672,
 (10, 200): 0.7439829701073973,
 (10, 225): 0.744038844749321,
 (10, 250): 0.7439328473253523,
 (10, 275): 0.7437808981988887,
 (10, 300): 0.7436279679310994,
 (15, 150): 0.7384151929600384,
 (15, 175): 0.7395745667758291,
 (15, 200): 0.7397775241624421,
 (15, 225): 0.7406348641320447,
 (15, 250): 0.7412356009567348,
 (15, 275): 0.7418715424502969,
 (15, 300): 0.7420645723626871}

In [74]:
model_rfc = RandomForestClassifier(random_state=42, n_estimators=300, max_depth=15)
model_gbc = GradientBoostingClassifier(random_state=42, n_estimators=225, max_depth=10)

In [75]:
model_rfc.fit(train_df[bemeno_valtozok], train_df['TARGET'])
model_gbc.fit(train_df[bemeno_valtozok], train_df['TARGET'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=225,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [77]:
prediction_rfc = model_rfc.predict_proba(test_df[bemeno_valtozok])[:,1]
prediction_gbc = model_gbc.predict_proba(test_df[bemeno_valtozok])[:,1]

In [78]:
a = np.array([prediction_rfc, prediction_gbc])

In [79]:
test_df['p1'] = np.mean(a, axis=0)

In [80]:
submission_df = test_df[['ID', 'p1']]
submission_df.columns = ['Id', 'Predicted']
submission_df.to_csv('proba6.csv', index=False)